# Tutorial

This tutorial shows the steps needed to apply dynamic fuzzing to a Plain Old Python Class (POPC).  Fuzzing is applied at the method level, although more sophisticated fuzzers can be constructed which filter out parts of a method's syntax tree as desired.

Consider the following Python class, <code>Hand</code> for representing the cleanliness state of your hands:

In [21]:
class Hands(object):
    def __init__(self):
        self.clean = False
        self.soaped = False

Now consider the classes <code>GetDirtyWorkflow</code>, <code>WashWorkflow</code> and <code>RinseWorkflow</code> describing workflows in [Theatre_Ag]() for washing and rinsing, respectively.

In [31]:
class GetDirtyWorkflow(object):
    
    is_workflow = True
    
    def __init__(self, washable):
        self.washable = washable
        
    def dirty(self):
        self.washable.clean = False
        self.washable.soaped = False

class RinseWorkflow(object):

    is_workflow = True

    def __init__(self, washable):
        self.washable = washable

    def rinse(self):
        self.washable.soaped = False


class WashWorkflow(object):

    is_workflow = True

    def __init__(self, washable):
        self.washable = washable
        self.rinse = RinseWorkflow(washable)

    def add_soap(self):
        self.washable.soaped = True

    def scrub(self):
        if self.washable.soaped:
            self.washable.clean = True

    def wash(self):
       self.add_soap()
       self.scrub()
       self.rinse.rinse()

We can fuzz the workflow so that the default sequence of steps in <code>wash</code> gets randomly permuted:

In [32]:
from pydysofu import fuzz_clazz
from pydysofu.core_fuzzers import shuffle_steps

advice = {
    WashWorkflow.wash: shuffle_steps
}
fuzz_clazz(WashWorkflow, advice)

The advice dictionary maps the function pointer to the <code>shuffle_steps</code> fuzzer.  The <code>fuzz_clazz</code> function then applies this advice dictionary to the whole <code>WashWorkflow</code> class.

Now we can use the fuzzed class as normal.

In [40]:
hands = Hands()
get_dirty_workflow = GetDirtyWorkflow(hands)
wash_workflow = WashWorkflow(hands)

for _ in range(0, 10):
    get_dirty_workflow.dirty()
    wash_workflow.wash()
    print hands.clean

False
False
True
False
False
False
False
True
False
False


Note that the fuzzer will be re-applied each time the fuzzed function is called meaning that in this case a different step can be removed from the workflow on each invocation.